In [1]:
import io
import PIL.Image
import numpy as np
from ipywidgets import Image, VBox, HBox, Widget, Button
import ipywidgets as widgets
from IPython.display import display
from ipywebrtc import CameraStream, ImageRecorder
from matplotlib import pyplot as plt
##########
def compress_to_bytes(data, fmt):
    """
    Helper function to compress image data via PIL/Pillow.
    """
    buff = io.BytesIO()
    img = PIL.Image.fromarray(data)    
    img.save(buff, format=fmt)
    return buff.getvalue()
##########

VIDEO_WIDTH = 420 # 窗口宽度，按需调整
VIDEO_HEIGHT = 320 # 窗口高度，按需调整

camera = CameraStream(constraints=
                      {'facing_mode': 'user',	
                       'audio': False,	
                       'video': { 'width': VIDEO_WIDTH, 'height': VIDEO_HEIGHT}	
                       })	# 另一种CameraStream创建方式，参考下文组件介绍部分
image_recorder = ImageRecorder(stream=camera)
out = Image(width=VIDEO_WIDTH, height=VIDEO_HEIGHT)

FLAG_STOP = False	# 终止标记

def cap_image(_):	# 处理ImageRecord抓取到的图片的过程
    if FLAG_STOP:
        return	# 停止处理
    im_in = PIL.Image.open(io.BytesIO(image_recorder.image.value))
    im_array = np.array(im_in)[..., :3]
    canvas = process(im_array)	# process是处理图像数组的函数，这里没写出来，各位按处理需要自己写即可
    ####
    canvas = cv2.cvtColor(canvas, cv2.COLOR_BGR2GRAY)
    canvas = cv2.GaussianBlur(canvas, (5, 5), 0)
    ####
    im_out = PIL.Image.fromarray(canvas)
    f = io.BytesIO()
    im_out.save(f, format='png')
    out.value = f.getvalue()
    image_recorder.recording = True	# 重新设置属性，使ImageRecorder继续抓取

# 注册抓取事件，参考我另一篇Blog：https://qxsoftware.blog.csdn.net/article/details/86708381

#image_recorder.image.observe(cap_image, names=['value'])

# 用于停止抓取的按钮
btn_stop = Button(description="Capture ok ?",
                  tooltip='click this to stop webcam',
                  button_style='danger')
# btn_stop的处理函数
def close_cam(_):

#     FLAG_STOP= True
#     Widget.close_all()
    im = PIL.Image.open(io.BytesIO(image_recorder.image.value))
    im_array = np.array(im)
    #################################
    fmt = 'png'
    data_comp = compress_to_bytes(im_array, fmt)
    print(type(data_comp))
    image_headline = widgets.Image(
                        value=data_comp,
                        format='jpg',
                        width='200'
                    )
    print(type(image_headline))
    label_headline = widgets.Label(
                        value='Photo by CDC on Unsplash',
                        style={'description_width': 'initial'}
                    )

    vbox_headline = widgets.VBox([image_headline, label_headline])
    hbox_headline = widgets.HBox([vbox_headline, vbox_headline])
    print(type(hbox_headline))
    # image_headline
    # print(type(vbox_headline))
    display(hbox_headline)
    #######################################################################""


btn_stop.on_click(close_cam) # 注册单击事件
# Run this section and Press the Camera button to display demo
display(VBox([HBox([camera, image_recorder  ]), btn_stop]))   #  btn_stop]), out]))
display(camera)

########
print("lets Go ! ")
#############


CameraStream(constraints={'facing_mode': 'user', 'audio': False, 'video': {'width': 420, 'height': 320}})

lets Go ! 


<class 'bytes'>
<class 'ipywidgets.widgets.widget_media.Image'>
<class 'ipywidgets.widgets.widget_box.HBox'>


In [2]:
Widget.close_all()